# Agents and Tools

#### ChatGPT Functions

In [181]:
import sqlite3
conn = sqlite3.connect('db.sqlite')

In [182]:
def run_query(query:str):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        return cursor.fetchall()
    except sqlite3.OperationalError as err:
        return f"The following error occurred: {str(err)}"
    
def describe_tables(tables_names):
    print("tables_names", tables_names)
    c = conn.cursor()
    tables = ', '.join("'" + table + "'" for table in tables_names)
    query = f"SELECT sql FROM sqlite_master WHERE type='table' and name in ({tables});"
    rows = c.execute(query) 
    formated = '\n'.join(row[0] for row in rows if row[0] is not None)
    return formated



## Making Custom Schema

In [183]:
from pydantic.v1 import BaseModel

class RunQueryArgsSchema(BaseModel):
    query: str

class DescribeTablesArgsSchema(BaseModel):
    tables_names: list[str]

In [184]:
from langchain.tools import Tool


run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="""
    run a query on the sqlite database
    """,
    func=run_query,
    args_schema=RunQueryArgsSchema,
)

describe_tables_tool = Tool.from_function(
    name="describe_tables",
    description="""
    Given a list of tables name, return the tables that are present in the database
    """,
    func=describe_tables,
    args_schema=DescribeTablesArgsSchema,
)

In [185]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from dotenv import load_dotenv
from langchain.schema import SystemMessage
load_dotenv()


True

In [186]:
def list_list_tables():
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    rows = c.fetchall() 
    return "\n".join(row[0] for row in rows if row[0] is not None)


In [187]:
chat = ChatOpenAI(
    model='gpt-3.5-turbo',
)

tables = list_list_tables()



chat_prompt = ChatPromptTemplate(
    messages =[
        SystemMessage(
            content=f"You are an AI that has access to SQLite database.\n The database has tables of:\n {tables}.\n Do not make any assumptions about the what tables exist or what colums exist. Instead use the 'describe_tables' function."
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [188]:
tools = [run_query_tool, describe_tables_tool]

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=chat_prompt,
    tools=tools,
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools,
)


In [189]:
agent_executor("How many users are in the database?")



> Entering new AgentExecutor chain...

Invoking: `describe_tables` with `{'tables_names': ['users']}`


tables_names ['users']
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT UNIQUE,
    password TEXT
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT COUNT(*) FROM users'}`


[(2000,)]There are 2000 users in the database.

> Finished chain.


{'input': 'How many users are in the database?',
 'output': 'There are 2000 users in the database.'}

In [190]:
agent_executor("How many users provided shipping address?")



> Entering new AgentExecutor chain...

Invoking: `describe_tables` with `{'tables_names': ['users', 'addresses']}`


tables_names ['users', 'addresses']
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT UNIQUE,
    password TEXT
    )
CREATE TABLE addresses (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    address TEXT
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT COUNT(DISTINCT user_id) FROM addresses'}`


[(2000,)]There are 2000 users who provided a shipping address.

> Finished chain.


{'input': 'How many users provided shipping address?',
 'output': 'There are 2000 users who provided a shipping address.'}

## Report maker

In [191]:
from langchain.tools import StructuredTool

In [192]:
class ReportHTMLArgsSchema(BaseModel):
    filename: str
    html: str

In [193]:
def write_report_html(filename:str, html:str):
    with open(filename, "w") as f:
        f.write(html)
    return f"Report saved to {filename}"

In [195]:
write_report_tool = StructuredTool.from_function(
    name="write_report",
    description="Write and HTML file to disk. use this tool whenever asked for a report",
    func=write_report_html,
    args_schema=ReportHTMLArgsSchema
)

In [196]:
tools_with_html = tools + [write_report_tool]


html_agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=chat_prompt,
    tools=tools_with_html,
)

html_agent_executor = AgentExecutor(
    agent=html_agent,
    verbose=True,
    tools=tools_with_html,
)



In [198]:

html_agent_executor("Summerize top 5 most popular products and save the report to 'report.html'")



> Entering new AgentExecutor chain...

Invoking: `describe_tables` with `{'tables_names': ['products', 'order_products']}`


tables_names ['products', 'order_products']
CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    price REAL
    )
CREATE TABLE order_products (
    id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    amount INTEGER
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT product_id, SUM(amount) as total_amount FROM order_products GROUP BY product_id ORDER BY total_amount DESC LIMIT 5'}`


[(3929, 62), (1657, 55), (866, 55), (1437, 53), (3192, 52)]
Invoking: `describe_tables` with `{'tables_names': ['products']}`


tables_names ['products']
CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    price REAL
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT * FROM products WHERE id IN (3929, 1657, 866, 1437, 3192)'}`


[(866, 'Ball', 303.47), (1437, 'Refined Chips', 468.43), (1657, 'Rubber Mouse', 4

{'input': "Summerize top 5 most popular products and save the report to 'report.html'",
 'output': "I have summarized the top 5 most popular products and saved the report to 'report.html'. You can download the report [here](sandbox:/report.html)."}